In [1]:
include("load_pod.jl");
include("load_s1slc_ann.jl");
include("llh2sali.jl")

llh2sali (generic function with 1 method)

In [2]:
llh = [
    [(56 + 33/60 + 42.5556/(60*60)),(8 + 18/60 + 26.874/(60*60)), 42.753],
    [(56 + 33/60 + 44.4528/(60*60)) ,(8 + 18/60 + 24.865/(60*60)),43.282],
    [(56 + 33/60 + 42.8652/(60*60)) ,(8 + 18/60 + 22.104/(60*60)),44.475],
    [(56.003092) ,8.129702 ,43.5]
];
llh_cat = vcat(llh'...);

In [3]:
f = open("ann_path.txt")
path = readlines(f)
sar_parameters = load_s1slc_ann(path[1]);
f = open("POD_path.txt")
path = readlines(f)
osv,t_sv = load_pod(path[1],sar_parameters["t_0"]);

In [4]:
sali = llh2sali(llh_cat, osv, t_sv, sar_parameters)

4×2 Array{Float64,2}:
 1126.43  6086.27
 1122.66  6098.7 
 1126.71  6109.43
 5685.96  6255.93

In [5]:
salih = hcat(sali,llh_cat[:,3])

4×3 Array{Float64,2}:
 1126.43  6086.27  42.753
 1122.66  6098.7   43.282
 1126.71  6109.43  44.475
 5685.96  6255.93  43.5  

In [9]:
include("calc_sat_trajectory.jl");
include("approx_los.jl");
include("intersect.jl")
include("xyz2ellh.jl");
include("radar_solve.jl");

In [26]:
function salih2llh(salih, osv, t_sv, sar_parameters)
    c = 299792458 # speed of light
    t_start = sar_parameters["t_start"]
    t_stop = sar_parameters["t_stop"]
    sign_angle  = sar_parameters["right_looking"] ? 1 : -1
    theta_0 = sign_angle*abs(sar_parameters["incidence_angle_mid"]*pi/180) 
    range_pixel_spacing =  sar_parameters["range_pixel_spacing"]
    inv_azimuth_frequency =  1/sar_parameters["azimuth_frequency"]
    r_near =  sar_parameters["slant_range_time"]  *c/2
    
    osv_poly, osv_mean, osv_std = calc_sat_trajectory(osv, t_sv, t_start, t_stop)
    rad2deg = 180/pi
    
    llh = Array{Float64}(undef,size(salih)[1],3)
    
        
    for i in 1:size(salih)[1]
        time =  t_start + (salih[i,1]-1)*inv_azimuth_frequency
        range_x = r_near + (salih[i,2] - 1)*range_pixel_spacing
            
        osv_0 = polyval_sv(osv_poly,time,osv_mean, osv_std)
        x_sat = osv_0[1:3]
        v_sat = osv_0[4:6]
            
        x_0 = intersect(x_sat, approx_los(x_sat,v_sat,theta_0))
        x = radar_solve(range_x,salih[i,3],x_0,x_sat,v_sat) 
        point =  xyz2ellh(x...)
        llh[i,1] = point[1]*rad2deg
        llh[i,2] = point[2]*rad2deg
        llh[i,3] =point[3]
        
    end
    
    return llh

end
    
    

salih2llh (generic function with 1 method)

In [27]:
llh_test = salih2llh(salih, osv, t_sv, sar_parameters)

4×3 Array{Float64,2}:
 56.5618  8.30747  42.7529
 56.5623  8.30691  43.2819
 56.5619  8.30614  44.4749
 56.0031  8.1297   43.4999

In [28]:
llh_cat .- llh_test

4×3 Array{Float64,2}:
 -3.27347e-8  -1.0865e-8   5.10481e-5
  1.06273e-8   2.29406e-9  5.16776e-5
  3.46767e-8   9.57187e-9  5.31044e-5
 -1.78345e-8  -6.25741e-9  5.27855e-5